In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=false; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [6]:
import os
import pandas as pd

# Imports the Google Cloud client library
from google.cloud import storage


def get_blobs(bucket_name, startPrefix, endPrefix):
    """Lists all the blobs in the bucket."""
    storage_client = storage.Client()
    
    # Note: Client.list_blobs requires at least package version 1.17.0. 
    blobs = storage_client.list_blobs(bucket_name, 
                                      start_offset=startPrefix,
                                      end_offset=endPrefix)
    
    retBlobs = []
    for blob in blobs:
        retBlobs.append(f'gs://{bucket_name}/{blob.name}')
        
    return retBlobs
        
sBucketUrl='ebfa_fixtures'

# Set GOOGLE_APPLICATION_CREDENTIALS to the private key for GCP
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/JK-MBPro/Documents/github.com/caversham.afc/key/CavershamAFC-1e2c4f66b68b.json"

# it is mandatory initialize the storage client
blobs = get_blobs(sBucketUrl, '4389-7590-2020-12-19', '4389-7590-2020-12-31')

dfUpdates = pd.DataFrame()

for blob in blobs:
    dfUp = pd.read_csv(blob, dtype=str)
    dfUpdates = pd.concat([dfUpdates,dfUp], axis=0, ignore_index=True)

#dfUpdates.head(20)

In [7]:
if 'HG_old' in dfUpdates.columns:
    printmd('<br>**Score updates:**')
    # drop all change columns and Updated and Notes columns
    dfScores = dfUpdates.loc[dfUpdates.HG.notnull()].copy()
    dfScores.drop('Updated', axis=1, inplace=True)
    dfScores.drop('Notes', axis=1, inplace=True)
    for col in dfScores.columns:
        if '_' in col:
            dfScores.drop(col, axis=1, inplace=True)
    display(dfScores.head())

<br>**Score updates:**

,MatchId,Date,Home,HG,AG,Away
1,4830,2020-12-19,HAWLEY YOUTH RAIDERS,0,1,WHITEKNIGHTS


In [8]:
pd.set_option('display.max_colwidth', 100)

# get every second row, i.e. just the row with new values
dfOtherUpdates = dfUpdates.iloc[1::2,:].copy()

# drop change columns of scores
for col in dfOtherUpdates.columns:
    if 'HG_' in col:
        dfOtherUpdates.drop(col, axis=1, inplace=True)
    if 'AG_' in col:
        dfOtherUpdates.drop(col, axis=1, inplace=True)

# drop rows that have no changes (i.e. null in all change columns)
cols = [col for col in dfOtherUpdates if (col.endswith('_old') | col.endswith('_new'))]
dfOtherUpdates = dfOtherUpdates.dropna(subset=cols, how='all')

# output only if there is anything
if dfOtherUpdates.size > 0:
    printmd('<br>**Fixtures updates:**')
    display(dfOtherUpdates)

<br>**Fixtures updates:**

,MatchId,Updated,Date,Home,HG,AG,Away,Notes,Notes_old,Notes_new
3,4827,2020-12-20T120509,2020-12-19,CAVERSHAM AFC JAGUARS,NaN,NaN,ROTHERFIELD UNITED WASPS,Match# 4827 | postponed due to weather 19/12 | Postponed,Match# 4827,Match# 4827 | postponed due to weather 19/12 | Postponed
5,4826,2020-12-20T150510,2020-12-19,REEVES RANGERS BLUES,NaN,NaN,WINDSOR YOUTH EAGLES TIGERS,Match# 4826 | postponed due to weather 19/12 | Postponed,Match# 4826,Match# 4826 | postponed due to weather 19/12 | Postponed
7,4828,2020-12-20T150510,2020-12-19,AFC CHARVIL,NaN,NaN,HEARTS OF TEDDLOTHIAN CHEETAHS,Match# 4828 | postponed due to weather 19/12 | Postponed,Match# 4828,Match# 4828 | postponed due to weather 19/12 | Postponed
